#Про корпус

Мой корпус - собрание нескольких предложений, в которых содержатся трудные случаи для разметки PoS-тэгов. В основном туда входят случаи, где в разных контекстах одни и те же слова относятся к разным частям речи. Например:

*Один (сущ.) в поле не воин.*

*Диалекты слились в один (прил.) язык.*

*В один (мест.) прекрасный день.*

Примеры были взяты из справочников и грамматик. [Отсюда](https://pandia.ru/text/78/356/1392.php#:~:text=%D0%A1%D0%BB%D0%B5%D0%B4%D1%83%D0%B5%D1%82%20%D0%B8%D0%BC%D0%B5%D1%82%D1%8C%20%D0%B2%20%D0%B2%D0%B8%D0%B4%D1%83%2C%20%D1%87%D1%82%D0%BE,%D0%BE%D0%B4%D0%BD%D0%BE%D0%B9%20%D1%87%D0%B0%D1%81%D1%82%D0%B8%20%D1%80%D0%B5%D1%87%D0%B8%20%D0%B2%20%D0%B4%D1%80%D1%83%D0%B3%D1%83%D1%8E.&text=%D0%92%20%D1%81%D0%BE%D0%B2%D1%80%D0%B5%D0%BC%D0%B5%D0%BD%D0%BD%D0%BE%D0%BC%20%D1%80%D1%83%D1%81%D1%81%D0%BA%D0%BE%D0%BC%20%D1%8F%D0%B7%D1%8B%D0%BA%D0%B5%20%D0%BC%D0%BE%D0%B6%D0%BD%D0%BE%20%D0%B2%D1%8B%D0%B4%D0%B5%D0%BB%D0%B8%D1%82%D1%8C%20%D1%83%D0%B7%D1%83%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5%20%D0%B8%20%D0%BA%D0%BE%D0%BD%D1%82%D0%B5%D0%BA%D1%81%D1%82%D1%83%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5%20%D0%BE%D0%BC%D0%BE%D0%BD%D0%B8%D0%BC%D0%B8%D1%87%D0%BD%D1%8B%D0%B5%20%D1%87%D0%B0%D1%81%D1%82%D0%B8%20%D1%80%D0%B5%D1%87%D0%B8.) и [отсюда](https://videotutor-rusyaz.ru/uchenikam/teoriya/65-omonimiyaslov.html). А также некоторые я придумала сама, доволнив примеры словосочетаний контекстом для образования цельных предложений. Поэтому вопросов по поводу, каким PoS-тегом должен быть размечен каждый токен не возникло.

#Про тегеры

Для анализа я выбрала бибилиотеки spacy, stanza, natasha. Все они используют тег-сет UD, поэтому необходимости переводить теги в единую систему нет. Для разметки своего корпуса я пользовалась этим же тегсетом.


#Работа с тегами

Сначала установим все необходимые бибилиотеки и напишем функции для разметки тегов.

###Spacy

In [ ]:
!pip install spacy==3.1

In [ ]:
!python -m spacy download ru_core_news_sm

In [3]:
import spacy
from spacy.lang.ru.examples import sentences 

nlp_spacy = spacy.load("ru_core_news_sm")

In [72]:
def pos_spacy(sent):
    doc = nlp_spacy(sent)
    return [(token.text, token.pos_) for token in doc if token.pos_ != 'SPACE']

###Natasha

In [ ]:
!pip install natasha

In [5]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

In [6]:
>>> segmenter = Segmenter()
>>> morph_vocab = MorphVocab()

>>> emb = NewsEmbedding()
>>> morph_tagger = NewsMorphTagger(emb)
>>> syntax_parser = NewsSyntaxParser(emb)
>>> ner_tagger = NewsNERTagger(emb)

>>> names_extractor = NamesExtractor(morph_vocab)

In [46]:
def pos_natasha(sent):
    doc = Doc(sent)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    return [(x.text, x.pos) for x in doc.tokens]

###Stanza

In [ ]:
!pip install stanza

In [ ]:
import stanza
stanza.download('ru') 
nlp = stanza.Pipeline('ru')

In [44]:
def pos_stanza(sent):
    doc = nlp(sent)
    return [(word.text, word.upos) for sent in doc.sentences for word in sent.words]

#Анализ

Теперь прочитаем тексты корпуса: один чистый, другой размеченный вручную.

In [7]:
import re

In [88]:
with open('marked.txt', encoding='utf-8') as file:
    PoS_true = []
    for line in file:
        PoS_true.extend(re.findall('([а-яА-ЯёЁ.?,-]+)_(\w+)', line))

In [78]:
with open('corpora.txt', encoding='utf-8') as file:
    text = file.read().replace('\n', ' ')

Разметим копус тегерами

In [79]:
PoStags_spacy = pos_spacy(text)
PoStags_natasha = pos_natasha(text)
PoStags_stanza = pos_stanza(text)

Проверим на совпадение количества токенов.

In [89]:
len(PoStags_spacy), len(PoS_true), len(PoStags_natasha), len(PoStags_stanza)

(256, 254, 254, 254)

У spacy получилось больше токенов, чем должно быть, потому что он делит слова с дефисом на несколько токенов. Для того, чтобы можно было сравнивать его работу с золотым стандартом, напишем функцию, которая объединяет такие токены в один.

In [95]:
def correct_spacy(tokens):
    i = 0
    while i < len(tokens):
        if tokens[i][0] == '-':
            new = tokens[:i-1]
            correct_token = (tokens[i-1][0]+tokens[i][0]+tokens[i+1][0], tokens[i][1])
            new.append(correct_token)
            new.extend(tokens[i+2:])
            tokens = new
            i += 1
        i += 1
    return tokens

In [98]:
PoStags_spacy = correct_spacy(PoStags_spacy)

In [100]:
len(PoStags_spacy), len(PoS_true)

(254, 254)

Теперь все хорошо. Далее напишем функцию для измерения качества работы библиотек.

In [110]:
def count_accuracy(true, check):
    metric = 0
    for i in range(len(true)):
        if check[i][1] == true[i][1]:
            metric += 1
    return metric / len(true)

In [112]:
print(f'spacy: {count_accuracy(PoS_true, PoStags_spacy)}')
print(f'natasha: {count_accuracy(PoS_true, PoStags_natasha)}')
print(f'stanza: {count_accuracy(PoS_true, PoStags_stanza)}')

spacy: 0.8779527559055118
natasha: 0.8622047244094488
stanza: 0.8740157480314961


В целом, качество разнится не сильно. Проблемы возникают как раз при разметке омонимичных слов. Лучше всех справился spacy, хоть и с небольшим отрывом. 

(На самом деле, я не скажу, что мой корпус очень сбалансирован по случаям. Поэтому качество зависит еще и от того, каки случаев у меня больше. Например, если предложений, когда причастие переходит в прилагательное, могло оказаться больше, чем тех, где прилагательное - в существительное, то тегер, который лучше справляется с первой неоднозначностью, покажет качество выше, чем тот, который, наоборот, лучше справляется со вторым случаем омонимичности).

#Chunker

In [114]:
def chunker(tokens):
    adj_noun = []
    verb_noun = []
    verb_adp_noun = []
    i = 0
    while i < len(tokens):
        if tokens[i][1] == 'ADJ' and tokens[i+1][1] == 'NOUN':
            adj_noun.append((tokens[i][0] + ' ' + tokens[i+1][0]))
            i += 1
        elif tokens[i][1] == 'VERB':
            if tokens[i+1][1] == 'NOUN':
                verb_noun.append((tokens[i][0] + ' ' + tokens[i+1][0]))
                i += 1
            elif tokens[i+1][1] == 'ADP' and tokens[i+2][1] == 'NOUN':
                verb_adp_noun.append((tokens[i][0] + ' ' + tokens[i+1][0] + ' ' + tokens[i+2][0]))
                i += 2
        i += 1
    return adj_noun, verb_noun, verb_adp_noun

In [123]:
a_n, v_n, v_a_n = chunker(PoStags_spacy)
print(f'Прилагательное + существительное:\n{a_n}\n')
print(f'Глагол + существительное:\n{v_n}\n')
print(f'Глагол + предлог + существительное:\n{v_a_n}')

Прилагательное + существительное:
['прекрасный день', 'южном тропике', 'темном лесе', 'красивые ели', 'ванную комнату', 'образованные люди']

Глагол + существительное:
['захватили штили', 'ели суп', 'кончилась ничьей', 'блестящий оратор', 'горящие дрова', 'горящие глаза', 'образованная взрывом', 'отдал сапоги', 'говори врастяжку']

Глагол + предлог + существительное:
['сидела в ванной', 'согласились на ничью', 'смотрели на иней', 'блестящий на солнце', 'собираться у камина', 'верится в правду']


#Задание 5

Если честно, я не поняла идею, чем может помочь добавление н-грам... 

Я добавила шаблоны:

- прилагательное + существительное

- наречие + прилагательное

- наречие + прилагательное + существительное

Эти н-грамы, вероятно, относятся к описательным, то есть передадут сентиментальную окраску комментариев.

Сам код добавлен в конец тетрадки с предыдущей ДЗ.

Качество упало. (опять повторюсь, я не поняла смысла этой идеи, как она могла улучшить качество?)